In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
#リスト8-29:GANの生成者ネットワーク
import keras
from keras import layers
import numpy as np

latent_dim = 32 #最初に設定する潜在空間ベクトル次元
height = 32 #縦32ピクセル
width = 32 #横32ピクセル
channels = 3 #色3チャネル

#入力変数(inputは正規分布のランダムノイズ)
generator_input = keras.Input(shape=(latent_dim,))

#入力を16×16、128チャネルの特徴マップに変換
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x) #活性化関数は"LeakyReLU"を使用
x = layers.Reshape((16, 16, 128))(x)

#畳み込み層を追加(畳込みカーネルサイズは5)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

#32×32にアップサンプリング(畳込みカーネルサイズは4)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

#さらに畳み込み層を追加(畳込みカーネルサイズは5)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

#32×32、3(channels)チャネル（CIFAR10の画像の形状）の特徴マップを生成(畳込みカーネルサイズは7)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x) #最後の活性化関数は"tanh"を使用

#形状がlatent_dim(32)の入力を、形状がx(32,32,3)の画像にマッピング
generator = keras.models.Model(generator_input, x) #"generator"に生成者ネットワークを格納
generator.summary()

Using TensorFlow backend.


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048

In [3]:
#リスト8-30:GANの判別者ネットワーク

#入力変数(shapeは(縦32,横32,カラーチャネル3))
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

#ドロップアウト層を１つ追加:重要なトリック!
x = layers.Dropout(0.4)(x)

#全結合分類層(2値分類)
x = layers.Dense(1, activation='sigmoid')(x)

#disctriminatorモデルをインスタンス化:形状が(32,32,3)の入力で2値分類(fake/real)を実行
discriminator = keras.models.Model(discriminator_input, x) #"discriminator"に判別者ネットワークを格納
discriminator.summary()

#オプティマイザで勾配刈込を使用し（clipvalue）、訓練を安定させるために学習率減衰を使用(decay)
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008,
                                                   clipvalue=1.0, #（勾配の）絶対値が1.0を超えた時、勾配がクリップ（切り取り）されます(勾配爆発を防ぐ)
                                                   decay=1e-8) #各更新の学習率減衰
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 6, 6, 128)         0   

In [4]:
#リスト8-31:敵対者ネットワーク(生成者と判別者をつなぎ合わせる)
discriminator.trainable = False #discriminatorを凍結するため、重みの更新不可能に設定(これはganモデルにのみ適用される)

gan_input = keras.Input(shape=(latent_dim,)) #gan_inputに生成者ネットワークの入力(正規分布のランダムノイズ)
gan_output = discriminator(generator(gan_input)) #gan_outputに判別者ネットワークの出力(2値分類)
gan = keras.models.Model(gan_input, gan_output) #"gan"に生成者ネットワークのランダムノイズを入力し、判別者ネットワークでの2値分類を出力するモデルを格納

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [5]:
#リスト8-32:GANの訓練の実装
from keras.preprocessing import image
from tensorflow.keras.datasets import cifar10 #cifar10を読み込むため追加したコード

(x_train, y_train), (_, _) = cifar10.load_data() #cifar10を読み込むため追加したコード

#カエルの画像（クラス６）を選択
x_train = x_train[y_train.flatten() == 6]

#データを正規化
x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20

#生成された画像の保存先を指定
save_dir = '8gan' #cifar10データセットを学内サーバで使用するため追加したコード


############################## 学習 #################################
start = 0
for step in range(iterations): #訓練ループを開始(10000回)
    
    #dircriminatorモデルの訓練用に潜在空間から点をランダムに抽出➀(正規分布のランダムノイズ, shapeは(20,32))
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    #generatorモデルでランダムノイズ(20,32)を形状(20,32,32,3)の画像にマッピング
    generated_images = generator.predict(random_latent_vectors)

    #本物の画像と組み合わせる
    stop = start + batch_size #一回につきバッチサイズ20ずつ足す
    real_images = x_train[start: stop] #一回につきバッチサイズ20ずつ本物の画像を抽出(形状を偽物の(20,32,32,3)に合わせる)
    combined_images = np.concatenate([generated_images, real_images]) #本物と偽物の画像を連結する(形状が(40,32,32,3))

    #本物の画像と偽物の画像を区別するラベルを立てる。　combined_imagesデータに合わせ、最初の20個の偽物は1ベクトル×20(20,1)、
    #残りの20個の本物は0ベクトル×20(20,1)とし、それらを連結して正解ラベル×40(40,1)を作る)
    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    
    #ラベルにランダムノイズを追加:重要なトリック！
    labels += 0.05 * np.random.random(labels.shape)

    #"discriminator"モデルを訓練(.train_on_batchで単一のバッチを使用して1回だけトレーニングする)
    d_loss = discriminator.train_on_batch(combined_images, labels) #入力には本物、偽物の画像を連結した画像データ、出力には同様の正解ラベル

    #dircriminatorモデルの訓練用に潜在空間から点をランダムに抽出➁(正規分布のランダムノイズ, shapeは(20,32))
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    #「これらはすべて本物の画像」であることを示すラベルを立てる(本物なので0ベクトル×20(20,1))
    misleading_targets = np.zeros((batch_size, 1))

    #"gan"モデルを通じてgeneratorを訓練(ganモデルではdiscriminatorの重みが凍結される)
    #訓練データの入力は偽物画像（ランダムノイズ）だが、出力(目的値)は全て本物の画像(0ベクトル)であるとして
    #嘘を付いた形の訓練をすることで、偽物画像が本物画像に近づく変換をするモデルに適合していく
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

    #訓練1回終わったら次の訓練では今回使用したものの20枚後の画像データを使用
    start += batch_size
    
    #訓練に全てのカエル画像データ5000枚を使用し終えたら(x_train[4980:5000])、また最初の20枚の訓練データを使用して訓練(x_train[0:20])
    if start > len(x_train) - batch_size:
      start = 0
    
    ##################### 訓練100回おきに保存とプロット ######################
    if step % 100 == 0:
        gan.save_weights('gan.h5') #モデルの重みを保存

        #"discriminator"および"gan"モデルの訓練誤差(binary_crossentropy)を出力
        print('discriminator loss:', d_loss)
        print('adversarial loss:', a_loss)

        #生成された画像を1つずつ保存(generated_frog0.ping, generated_frog100.ping,..., generated_frog9900.ping)
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))

        #比較のために本物の画像を1つずつ保存(real_frog0.ping, real_frog100.ping,..., real_frog9900.ping)
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png'))

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss: 0.7049017
adversarial loss: 0.6673985


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss: 0.74159485
adversarial loss: 0.9378754
discriminator loss: 0.70603085
adversarial loss: 0.8391715
discriminator loss: 0.7147874
adversarial loss: 0.7847269
discriminator loss: 0.67453337
adversarial loss: 0.7909132
discriminator loss: 0.6900134
adversarial loss: 0.79012215
discriminator loss: 0.6988419
adversarial loss: 0.7405921
discriminator loss: 0.68553
adversarial loss: 0.75800526
discriminator loss: 0.70742035
adversarial loss: 0.713962
discriminator loss: 0.69843376
adversarial loss: 0.73864704
discriminator loss: 0.69773877
adversarial loss: 0.80400944
discriminator loss: 0.6914603
adversarial loss: 0.70939744
discriminator loss: 0.6990951
adversarial loss: 0.7305231
discriminator loss: 0.6936127
adversarial loss: 0.7225195
discriminator loss: 0.68484116
adversarial loss: 0.73860776
discriminator loss: 0.7216835
adversarial loss: 0.82687837
discriminator loss: 0.6998333
adversarial loss: 0.72169316
discriminator loss: 0.71403235
adversarial loss: 0.74646825
